In [1]:
import os
import time
import torch
import zipfile
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
import torchvision.models as models

from torch.cuda import amp
from PIL import Image, ImageFile
from torchvision import transforms
from torchvision.io import read_image
from tqdm.notebook import tqdm, trange
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_score, recall_score, accuracy_score

ImageFile.LOAD_TRUNCATED_IMAGES = True

import warnings
warnings.filterwarnings('ignore')

In [2]:
MEAN = torch.tensor((0.485, 0.456, 0.406))
STD  = torch.tensor((0.229, 0.224, 0.225))
BATCH_SIZE = 64
NUM_CLASSES = 2 # Healthy & Not Healthy
TRAIN_DATA_PATH = "/kaggle/input/binary-coco-first-article/train"
TEST_DATA_PATH = "/kaggle/input/binary-coco-first-article/test"

In [3]:
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

In [4]:
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)


# Train
dataloader_train = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# Test
dataloader_test = torch.utils.data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

# Validation
dataloader_valid = torch.utils.data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class EfficientNetB1(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB1, self).__init__()
        self.effnet = models.efficientnet_b1(pretrained=True)
        num_ftrs = self.effnet.classifier[1].in_features
        self.effnet.classifier[1].fc = nn.Linear(num_ftrs, NUM_CLASSES)

    def forward(self, x):
        return self.effnet(x)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [7]:
model = EfficientNetB1(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 103MB/s]


In [8]:
def train_model(learningRate, num_epochs):
  # Define Adam optimizer with the current learning rate
  optimizer = optim.SGD(model.parameters(), lr=learningRate)

  # Training loop
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0
      all_labels = []
      all_predictions = []
      for inputs, labels in dataloader_train:
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()

          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item() * inputs.size(0)

          _, predicted = torch.max(outputs, 1)
          all_labels.extend(labels.cpu().numpy())
          all_predictions.extend(predicted.cpu().numpy())

      # Calculate average training loss per epoch
      epoch_loss = running_loss / 1800 # len(train_data)
      precision = precision_score(all_labels, all_predictions, average='weighted')
      recall = recall_score(all_labels, all_predictions, average='weighted')
      accuracy = accuracy_score(all_labels, all_predictions)
      # print(f"LR : {learningRate} - Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}")
  torch.save(model.state_dict(), f'model_{learningRate}.pth')
  # Evaluation loop
  model.eval()
  all_labels = []
  all_predictions = []
  with torch.no_grad():
      for inputs, labels in dataloader_valid:
          inputs, labels = inputs.to(device), labels.to(device)

          outputs = model(inputs)

          _, predicted = torch.max(outputs, 1)
          all_labels.extend(labels.cpu().numpy())
          all_predictions.extend(predicted.cpu().numpy())

  precision = precision_score(all_labels, all_predictions, average='weighted')
  recall = recall_score(all_labels, all_predictions, average='weighted')
  accuracy = accuracy_score(all_labels, all_predictions)
  print(f"Learning Rate: {learningRate}, Recall: {recall:.4f}, Precision: {precision:.4f}, Accuracy: {accuracy:.4f}")


In [9]:
# Define learning rates
learning_rates = [0.001, 0.005, 0.010, 0.020] # 0.030, 0.050, 0.08, 0.1, 0.15]

In [10]:
epochs = 20
optimizer = optim.SGD(model.parameters(), lr=0.005)

In [11]:
for lr in learning_rates:
  train_model(lr, epochs)

Learning Rate: 0.001, Recall: 0.7875, Precision: 0.8750, Accuracy: 0.7875
Learning Rate: 0.005, Recall: 0.9500, Precision: 0.9505, Accuracy: 0.9500
Learning Rate: 0.01, Recall: 0.9750, Precision: 0.9760, Accuracy: 0.9750
Learning Rate: 0.02, Recall: 0.9875, Precision: 0.9877, Accuracy: 0.9875


In [12]:
# Сохранение модели

torch.save(model, '/kaggle/working/effnet.pth')
torch.save(model.state_dict(), '/kaggle/working/effnet_weights.pth')